In [ ]:
import glob as g
import numpy as np
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
class ensemble :
    
    def __init__(self, data) :
        
        self.data=data 
        np.random.shuffle(self.data)
        self.x=data[:, 0:22500]
        self.y=data[:, 22500]
        
        return
    
    def create_model_obj(self) :
        
        self.clf1 = DecisionTreeClassifier(max_depth=10)
        self.clf2 = KNeighborsClassifier(n_neighbors=11)
        self.clf3 = SVC(gamma='scale', kernel='rbf', probability=True)
        self.clf4 = GaussianNB()
        self.clf5 = RandomForestClassifier(n_estimators=50, random_state=1)
        self.clf6 = LGBMClassifier(boosting_type='dart', objective='binary', \
                     learning_rate=0.01, num_leaves=240, subsample_for_bin=5000, \
                     class_weight={0:0.6, 1:0.4})
        self.clf7 = XGBClassifier(max_depth=100, learning_rate=0.01, objective='binary:logistic',\
                                 n_jobs=10, gamma=0.1, subsample=0.6)
        
        self.eclf = VotingClassifier(estimators=[('dt', self.clf1), ('knn', self.clf2), \
                                                 ('svc', self.clf3), ('nb', self.clf4), \
                                                 ('rf', self.clf5), ('lgb', self.clf6), ('xgb', self.clf7)], \
                                     voting='soft', weights=[2, 1, 2, 1, 2, 4, 3])
        
        return [self.clf1, self.clf2, self.clf3, self.clf4, self.clf5, self.clf6, self.clf7, self.eclf]
    
    def train_models(self):
        
        self.model_objs=self.create_model_obj()
        for i in range(len(self.model_objs)):
            self.model_objs[i]=self.model_objs[i].fit(self.x, self.y)
        
        return self.model_objs
    
    def cross_validation(self):
        
        self.clfs=self.train_models()
        self.model_names=['dt', 'knn', 'svc', 'nb', 'rf', 'lgb', 'xgb', 'ens']
        
        for clf, label in zip(self.clfs, self.model_names):
            self.scores=cross_val_score(clf, self.x, self.y, cv=5, scoring='accuracy')
            print('Accuracy : %0.2f (+/- %0.2f) [%s]' % (self.scores.mean(), self.scores.std(), label))
        
        return

In [ ]:
if __name__=='__main__':
    data=np.loadtxt('34552_sci_OK_data(without noise).csv', delimiter=',', dtype=np.float32)
    e=ensemble(data)
    e.cross_validation()